- DCGAN 성능 평가를 위한 Simple CNN
- Convolution 세팅 출처: https://cs230.stanford.edu/blog/handsigns/
- Accuracy와 함께 confusion matrix를 그려 recall 값을 확인한다.

In [191]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, SubsetRandomSampler
import torch.optim as optim
import random
from sklearn.metrics import confusion_matrix
import numpy as np
import pandas as pd
from skin_dataset import SKIN_CANCER_TRAIN, SKIN_CANCER

- 에폭 수, learning rate, 배치 사이즈 결정

In [283]:
epochs = 30
learning_rate = 0.001
batch_size = 64

- 데이터를 로드하고, SubsetRandomSampler를 이용해 validation set(training set의 15%)도 설정해준다.
- 데이터 크기를 확인한다.

In [284]:
sc_trn = SKIN_CANCER('skincancer', 'train')
sc_tst = SKIN_CANCER('skincancer', 'test')
trn_total_idx = list(range(len(sc_trn)))
trn_total_idx = random.sample(trn_total_idx, len(trn_total_idx))
trn_num = int(np.floor(0.85*len(sc_trn)))
trn_idx, val_idx = trn_total_idx[:trn_num], trn_total_idx[trn_num:]

In [285]:
len(trn_total_idx)

3429

In [286]:
trn_loader = DataLoader(sc_trn, sampler=SubsetRandomSampler(trn_idx), batch_size=batch_size, drop_last=True)
val_loader = DataLoader(sc_trn, sampler=SubsetRandomSampler(val_idx), batch_size=batch_size, drop_last=True)
tst_loader = DataLoader(sc_tst, batch_size=len(sc_tst), shuffle=True)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [287]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        #we define convolutional layers 
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size = 3, stride = 1, padding = 1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, stride = 1, padding = 1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3, stride = 1, padding = 1)
        self.bn3 = nn.BatchNorm2d(128)

        #2 fully connected layers to transform the output of the convolution layers to the final output
        self.fc1 = nn.Linear(in_features = 8*8*128, out_features = 128)
        self.fcbn1 = nn.BatchNorm1d(128)
        self.fc2 = nn.Linear(in_features = 128, out_features = 6)       
        self.dropout_rate = 0.2
        
    def forward(self, s):
        #we apply the convolution layers, followed by batch normalisation, 
        #maxpool and relu x 3
        s = self.bn1(self.conv1(s))        # batch_size x 32 x 64 x 64
        s = F.relu(F.max_pool2d(s, 2))     # batch_size x 32 x 32 x 32
        s = self.bn2(self.conv2(s))        # batch_size x 64 x 32 x 32
        s = F.relu(F.max_pool2d(s, 2))     # batch_size x 64 x 16 x 16
        s = self.bn3(self.conv3(s))        # batch_size x 128 x 16 x 16
        s = F.relu(F.max_pool2d(s, 2))     # batch_size x 128 x 8 x 8

        #flatten the output for each image
        s = s.view(-1, 8*8*128)  # batch_size x 8*8*128

        #apply 2 fully connected layers with dropout
        s = F.dropout(F.relu(self.fcbn1(self.fc1(s))), 
        p=self.dropout_rate, training=self.training)    # batch_size x 128
        s = self.fc2(s)                                     # batch_size x 6

        return F.log_softmax(s, dim=1)
net = Net()

- Loss는 cross entropy loss를, optimizer는 Adam을 사용한다.

In [288]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

In [289]:
for epoch in range(epochs):  # loop over the dataset multiple times
    
    net = net.to(device)
    net.train()
    trn_loss = []
    val_loss = []
    for i, data in enumerate(trn_loader, 0):
        
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        trn_loss.append(loss.item())
    
    net.eval()
    for i, data in enumerate(val_loader, 0):

        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        
        val_loss.append(loss.item())
    
    print('[{} / {}] trn loss: {:.4f} val loss: {:.4f}'.format(epoch + 1, epochs, np.mean(trn_loss), np.mean(val_loss)))
print('Finished Training')

[1 / 30] trn loss: 0.6960 val loss: 0.5614
[2 / 30] trn loss: 0.4480 val loss: 0.4340
[3 / 30] trn loss: 0.4106 val loss: 0.4274
[4 / 30] trn loss: 0.3739 val loss: 0.4164
[5 / 30] trn loss: 0.3588 val loss: 0.4447
[6 / 30] trn loss: 0.3175 val loss: 0.4369
[7 / 30] trn loss: 0.3010 val loss: 0.5532
[8 / 30] trn loss: 0.2845 val loss: 0.4572
[9 / 30] trn loss: 0.2354 val loss: 0.4660
[10 / 30] trn loss: 0.2295 val loss: 0.4199
[11 / 30] trn loss: 0.1979 val loss: 0.4010
[12 / 30] trn loss: 0.1710 val loss: 0.4152
[13 / 30] trn loss: 0.1666 val loss: 0.4865
[14 / 30] trn loss: 0.1497 val loss: 0.4571
[15 / 30] trn loss: 0.1240 val loss: 0.5246
[16 / 30] trn loss: 0.0984 val loss: 0.5709
[17 / 30] trn loss: 0.0896 val loss: 0.4838
[18 / 30] trn loss: 0.0988 val loss: 0.5224
[19 / 30] trn loss: 0.0784 val loss: 0.5922
[20 / 30] trn loss: 0.0670 val loss: 0.5746
[21 / 30] trn loss: 0.0647 val loss: 0.5521
[22 / 30] trn loss: 0.0736 val loss: 0.5539
[23 / 30] trn loss: 0.0652 val loss: 0.52

In [290]:
tst_loss = []
tst_acc = []
net = net.to(torch.device('cpu'))
net.eval()

Net(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=8192, out_features=128, bias=True)
  (fcbn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=128, out_features=6, bias=True)
)

In [291]:
for i, data in enumerate(tst_loader, 0):
    inputs, labels = data
    output = net(inputs)
    loss = criterion(output, labels)
    _, preds = torch.max(output.data, 1)
    acc = (preds == labels).sum() / len(output)
    
    tst_loss.append(loss.item())
    tst_acc.append(acc.item())

print('Test Accuracy: {:.2f} % Test Loss: {:.4f}'.format(np.mean(tst_acc)*100, np.mean(tst_loss)))

Test Accuracy: 80.76 % Test Loss: 0.6253


- Confusion matrix 생성

In [294]:
cm= confusion_matrix(labels, preds)
cmdf= pd.DataFrame(cm, index=set(labels.numpy()), columns=set(labels.numpy()))
cmdf = cmdf.rename(columns = {0: 'benign', 1: 'malignant'}, index={0: 'benign', 1: 'malignant'}, inplace = False); 
cmdf

,benign,malignant
benign,315,45
malignant,82,218


- Recall 값 계산

In [293]:
tn, fp, fn, tp = cm.ravel()
recall = tp / (tp + fn)
print('tn, fp, fn, tp:', (tn, fp, fn, tp), 'recall: ', np.round(recall, 4))

tn, fp, fn, tp: (315, 45, 82, 218) recall:  0.7267
